In [27]:
df.head()

,Unnamed: 0,etoiles,n_avis,date_avis,date_experience,Société,text_total,text_clean_sentences_neg,text_clean_sentences_pos,text_stop,text_clean_sentences_neg_stop,text_clean_sentences_pos_stop
0,80800,5,2.0,2023-08-02 07:32:51+00:00,2023-08-01,SOFINCO,Simple et rapide. na,NaN,Simple et rapide,Simple rapide .,NaN,Simple rapide
1,45069,5,2.0,2021-02-23 11:30:21+00:00,2021-02-23,Cofidis,Rapidité efficacité. Conseiller à l'écoute. Ré...,NaN,Rapidité efficacité. Conseiller à l' écoute. R...,Rapidité efficacité . Conseiller écoute . Répo...,NaN,Rapidité efficacité . Conseiller l ' écoute . ...
2,81625,1,2.0,2016-06-07 04:18:33+00:00,2016-06-07,SOFINCO,refus de 600 euros sans raison j'ai aucun créd...,refus de 600 euros sans raison j' ai aucun cré...,NaN,refus 600 euros raison aucun crédit 2100 euros...,refus 600 euros raison j ' aucun crédit 2100 e...,NaN
3,60859,5,1.0,2017-06-24 17:57:58+00:00,2017-06-24,Immoprt,"""Rapide et efficace. Conseillère à l'écoute, e...",NaN,Rapide et efficace. Conseillère à l' écoute ef...,"""Rapide efficace . Conseillère écoute , effica...",NaN,Rapide efficace . Conseillère l ' écoute effic...
4,80796,1,3.0,2023-08-02 10:29:49+00:00,2023-07-25,SOFINCO,Demande déposée le 25 juillet… tjrs…. Demande ...,Demande déposée le 25 juillet. Demande déposée...,J’ ai déjà fait 3 demandes de crédits chez vou...,Demande déposée 25 juillet … tjrs … . Demande ...,Demande déposée 25 juillet . Demande déposée 2...,J ’ 3 demandes crédits honores soldés


In [1]:
import pandas as pd
import spacy

df = pd.read_csv("df_sim_small.csv")
references = pd.read_csv('references_bag.csv')


nlp = spacy.load("fr_core_news_lg")
#stop_words = spacy.lang.fr.stop_words.STOP_WORDS


# load the df with references, full and stop word


In [39]:
import numpy as np
import warnings
warnings.filterwarnings("ignore") # pour ignorer les warnings dans la comparaison de vecteurs nulls

i = np.random.randint(0,199)


def allocate_lab(sim_score, y_pred, thresh):
    #print("thresh", thresh, "sim_score", sim_score)
    try:
        if max(sim_score) > thresh:
            y_pred.append(1)
            #print(max(sim_score), "over threshold", round(thresh,2), "y_pred = 1")
        else:
            y_pred.append(0) 
            #print(max(sim_score), "under threshold", round(thresh,2), "y_pred = 0")
    except:
        y_pred.append(0)
        #print("error, y_pred = 0")
    return y_pred


# review sentence vs reference sentence
def decision_sentiment(sentiment,label_of_interest, word_mode):
    # determining if using stop words or full version of reference texts
    if "stop" in word_mode:
        ref_bag = label_of_interest+"_stop"
    elif "all" in word_mode:
        ref_bag = label_of_interest
    else:
        print("error in word_mode?", word_mode)

    # determining if using sentiment filtered review sentences or all reviews sentences
    if sentiment == True:
        if "good" in label_of_interest:
            if "stop" in word_mode:
                reviews= "text_clean_sentences_pos_stop"
            else:
                reviews =  "text_clean_sentences_pos"
        elif "bad" in label_of_interest:
            if "stop" in word_mode:
                reviews =  "text_clean_sentences_neg_stop"
            else:
                reviews =  "text_clean_sentences_neg"
        else:
            print("mispelling in label_of_interest?", label_of_interest)

    elif sentiment == False:
        # for some reason the original, uncleaned punctuation versions give much better accuracy
        if "stop" in word_mode:
            reviews = "text_stop"
        else:
            reviews = "text_total"
            
    else :
        print("error: no sentiment / granularity found")

    return reviews, ref_bag

def review_sentence_vs_ref_sentence(i,df,label_of_interest, thresh,word_mode, sentiment=True):
    test_name = "review_sentence_vs_ref_sentence"
    #print("passing",test_name) 
    reviews,ref_bag = decision_sentiment(sentiment,label_of_interest, word_mode)
    ref_bag = nlp(references[ref_bag].iloc[0])
    y_pred = [] 
    sim_score = []
    try:
        review = nlp(df[reviews].iloc[i])
        review_sentences = review.sents
        for review_sentence in review_sentences: # take each review sentence
            if len(review_sentence) > 1:
                for sentence_exemple in ref_bag.sents:
                    if len(sentence_exemple) > 1:
                        temp_sim_score = round(sentence_exemple.similarity(review_sentence),2) # test similarity with whole reference bag
                        sim_score.append(temp_sim_score)
    except:
        #print("message null")
        sim_score.append(0.0)
    y_pred = allocate_lab(sim_score, y_pred, thresh) # if similarity failed, mark as 0 (empty review or not sentence aligned with label sentiment if sentiment = True)     
    return y_pred

# Comparing whole review vs whole reference bag, returns y_pred series
def review_vs_whole_ref_bag(i,df,label_of_interest, thresh,word_mode, sentiment=True):
    # establish the correct review granularity, review and reference pieces to integrate based on sentiment bool, stop_word (word_mode) bool and label of interest
    reviews,ref_bag = decision_sentiment(sentiment,label_of_interest, word_mode) 
    #print("for label", label_of_interest)
    #print("review colname", reviews)
    #print("reference bag colname", ref_bag)
    ref_bag = nlp(references[ref_bag][0]) # we convert the references once to nlp before the loop
    y_pred = []
    sim_score = []
    try:
        review = nlp(df[reviews].iloc[i])
        sim_score.append(round(ref_bag.similarity(review),2)) # compare it with the full reference bag
        allocate_lab(sim_score, y_pred, thresh) # if similarity score is > than threshold we append 1, otherwise: 0
    except: # if similarity test fails, we put 0 (in this case na or non text review)
        print("nan or error on message:", review)
        y_pred.append(0)
    return y_pred 
    

# review vs whole reference bag
sim = pd.DataFrame()
sim['c_bad_com'] = review_sentence_vs_ref_sentence(i,df,"c_bad_com", 0.73,"stopwords", sentiment=True)
sim['c_bad_efficacy'] = review_vs_whole_ref_bag(i,df,"c_bad_efficacy", 0.86, "allwords", sentiment=False)
sim['c_good_efficacy'] = review_sentence_vs_ref_sentence(i,df, "c_good_efficacy", 0.71,"stopwords", sentiment=True)
sim['c_good_com'] = review_sentence_vs_ref_sentence(i,df, "c_good_com", 0.68,"stopwords", sentiment=True)
sim['c_good_value'] = review_sentence_vs_ref_sentence(i,df, "c_good_value", 0.68,"stopwords", sentiment=True)
sim['c_bad_value'] = review_sentence_vs_ref_sentence(i,df, "c_bad_value", 0.69,"stopwords", sentiment=True)

sim = sim.astype("bool")

if sim['c_bad_com'][0]:
    if sim['c_good_com'][0]:
        b_com = "😐 une communication mitigée"
    else:
        b_com ="👎 une mauvaise communication"
else:
    if sim['c_good_com'][0]:
        b_com = "👍 une bonne communication"
    else:
        b_com= ""#"😐 une communication mitigée"

if sim['c_bad_value'][0]:
    if sim['c_good_value'][0]:
        b_value = "😐 un rendement économique mitigé"
    else:
        b_value = "👎 un mauvais rendement économique"
else:
    if sim['c_good_value'][0]:
        b_value = "👍 un bon rendement économique"
    else:
        b_value = ""#"😐 un rendement économique mitigé"

if sim['c_bad_efficacy'][0]:
    if sim['c_good_efficacy'][0]:
        b_efficacy = "😐 une efficacité mitigée"
    else:
        b_efficacy ="👎 est inefficace"
else:
    if sim['c_good_efficacy'][0]:
        b_efficacy = "👍 est efficace"
    else:
        b_efficacy =""# "😐 a une efficacité mitigée"

print(df.text_total[i])

if sim['c_bad_com'][0] + sim['c_good_com'][0] + sim['c_bad_value'][0] + sim['c_good_value'][0] + sim['c_bad_efficacy'][0] + sim['c_good_efficacy'][0]:
    print(f"l'avis considère que la banque {df.Société[i]} a {b_com} {b_value} {b_efficacy}")
else:
    print("pas de sentiment particulier détécté")
    
b_com,b_value,b_efficacy = "","",""
sim.head()

cette banque tue mon entreprise suite à…. cette banque tue mon entreprise suite à un disfonctionnements de mon compte il m'annonce qu'il vont clôturer mon compte sous 60 jour a compte du 08/08/2023 je ne conteste pas. je recoins un virement sur mon compte anytime j'effectue le virement sur mon nouveau compte lol il me le bloque pour document pas assez précis après 5 jours d'envoi de mail toujours aucune réactivité ni de virement alors que le document demander à été fournis le jour même cette banque est nul pour les professionnel
l'avis considère que la banque Anytime a 👎 une mauvaise communication  👎 est inefficace


,c_bad_com,c_bad_efficacy,c_good_efficacy,c_good_com,c_good_value,c_bad_value
0,True,True,False,False,False,False


In [33]:
sim['c_bad_com'][0] + sim['c_good_com'][0]

True

In [24]:
i

138